In [ ]:
import os

# The current working directory needs to be in explainable_TGCNN
print(os.getcwd())
os.chdir('..\\')
print(os.getcwd())

import math
import pandas as pd
import numpy as np
import tensorflow as tf
from src import create_fake_patients, whole_model_demographics_gradcam, graph_plot, ga, gc, utils

In [ ]:
second_TGCNN_layer = True
demo = True

include_drugs = True
max_timesteps=100

stride = 1
filter_size = 4
num_filters=16

run_name='hip_1999_to_one_year_advance_model'
years_in_advance = "5"

if include_drugs:
    max_event_codes = 518
else:
    max_event_codes = 512
hip_or_knee = 'hip'

# fake mapping dataframe for the ReadCodes and the corresponding descriptions
read_code_map_df = pd.read_csv('fake_read_code_descriptions.csv')

model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=16, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')

# Load in the filters from the model
with open(f'hip_1999_to_one_year_advance_model1_filter.npy', 'rb') as f:
    filters = np.load(f)
filters = tf.cast(filters, dtype=tf.float16)

num_pats = 5
max_act_filt_num = 10
cv_patients = create_fake_patients.create_fake_patient_df(num_pats, 99, max_event_codes)

filters2 = []

for w in filters:
    w_mean = np.mean(w).astype(np.float16)
    std_mean = np.std(w).astype(np.float16)
    noise = np.random.normal(loc=w_mean, scale=std_mean, size=w.shape).astype(np.float16)
    modified_w = (w + noise)  # Add the noise to the weights
    filters2.append(modified_w)

filters2 = np.array(filters2, dtype=np.float16)

In [ ]:
def remove_nans(lst):
    cleaned_list = [x for x in lst if not math.isnan(x)]
    return cleaned_list

filt_type = 'mean' # 'mean', 'median', 'max'
mse_diffs_mean, abs_diffs_mean = [], []
mse_diffs_median, abs_diffs_median = [], []
mse_diffs_max, abs_diffs_max = [], []
for pat_num in range(num_pats):
    if (pat_num % 50) == 0 and (pat_num !=0):
        print(f"{(((pat_num+1)/num_pats)*100):.2f}% Complete")
        mse_diffs_mean = remove_nans(mse_diffs_mean)
        abs_diffs_mean = remove_nans(abs_diffs_mean)
        mse_diffs_median = remove_nans(mse_diffs_median)
        abs_diffs_median = remove_nans(abs_diffs_median)
        mse_diffs_max = remove_nans(mse_diffs_max)
        abs_diffs_max = remove_nans(abs_diffs_max)
        print(f"MSE mean +- SD: {np.mean(mse_diffs_mean):.3f} $\pm$ {np.std(mse_diffs_mean):.3f}")
        print(f"Abs mean +- SD: {np.mean(abs_diffs_mean):.3f} $\pm$ {np.std(abs_diffs_mean):.3f}")
        
        print(f"MSE median +- SD: {np.mean(mse_diffs_median):.3f} $\pm$ {np.std(mse_diffs_median):.3f}")
        print(f"Abs median +- SD: {np.mean(abs_diffs_median):.3f} $\pm$ {np.std(abs_diffs_median):.3f}")
        
        print(f"MSE max +- SD: {np.mean(mse_diffs_max):.3f} $\pm$ {np.std(mse_diffs_max):.3f}")
        print(f"Abs max +- SD: {np.mean(abs_diffs_max):.3f} $\pm$ {np.std(abs_diffs_max):.3f}")
        
    input_3d, input_4d, demo_tensor, outcome, outcome_bin = utils.return_pat_from_df(cv_patients, max_event_codes, hip_or_knee, pat_num, max_timesteps)
    dense_tensor = tf.sparse.to_dense(input_3d)
    dense_tensor= tf.transpose(dense_tensor, perm=[2, 1, 0])
    dense_tensor = np.flip(dense_tensor, axis=0) # change the most recent events to be at the end rather than the start
    dense_tensor = tf.cast(dense_tensor, tf.float16)
    
    
    filters_4d = ga.make_filts_4d(filters, filter_size, max_event_codes)
    heatmap_df = pd.DataFrame()
    
    for i in range(0, num_filters):
        f = ga.get_and_reshape_filt(filters_4d, max_act_filt_num, 'single', i)
        edge_act_graph = ga.filt_times_pat(f, dense_tensor, filter_size, max_timesteps, stride)
        edges_df = ga.create_edges_df_ga(dense_tensor, edge_act_graph) 
        
        # Get the node positions for the graph
        pos_df = graph_plot.create_position_df_gc(edges_df)
        pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
        pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
        read_code_pos_df = ga.map_read_code_labels(pos_df, read_code_map_df)
        edge_pos_df = ga.create_edge_pos_df(edges_df, pos_df)
        
        heatmap_df[f'col_{i}'] = edge_pos_df['edge_weight_perc']
    
    # add the mean, median and max map values to the heatmap too
    
    row_medians = heatmap_df.iloc[:, :].median(axis=1)
    heatmap_df['col_32'] = row_medians
    
    row_means = heatmap_df.iloc[:, :-1].mean(axis=1)
    heatmap_df['col_33'] = row_means
    
    row_max = heatmap_df.iloc[:, max_act_filt_num-1]
    heatmap_df['col_34'] = row_max
    
    heatmap1_mean = heatmap_df['col_32'].to_numpy()
    heatmap1_median = heatmap_df['col_33'].to_numpy()
    heatmap1_max = heatmap_df['col_34'].to_numpy()

    # ---------------------------------------------------------------------------------------------

    filters2_4d = ga.make_filts_4d(filters2, filter_size, max_event_codes)
    heatmap_df2 = pd.DataFrame()
    
    for i in range(0, num_filters):
        f = ga.get_and_reshape_filt(filters2_4d, max_act_filt_num, 'single', i)
        edge_act_graph = ga.filt_times_pat(f, dense_tensor, filter_size, max_timesteps, stride)
        edges_df = ga.create_edges_df_ga(dense_tensor, edge_act_graph) 
        
        # Get the node positions for the graph
        pos_df = graph_plot.create_position_df_gc(edges_df)
        pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
        pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)
        read_code_pos_df = ga.map_read_code_labels(pos_df, read_code_map_df)
        edge_pos_df = ga.create_edge_pos_df(edges_df, pos_df)
        
        heatmap_df2[f'col_{i}'] = edge_pos_df['edge_weight_perc']
    
    # add the mean, median and max map values to the heatmap too
    
    row_medians = heatmap_df2.iloc[:, :].median(axis=1)
    heatmap_df2['col_32'] = row_medians
    
    row_means = heatmap_df2.iloc[:, :-1].mean(axis=1)
    heatmap_df2['col_33'] = row_means
    row_max = heatmap_df2.iloc[:, max_act_filt_num-1]
    heatmap_df2['col_34'] = row_max
    
    heatmap2_mean = heatmap_df2['col_32'].to_numpy()
    heatmap2_median = heatmap_df2['col_33'].to_numpy()
    heatmap2_max = heatmap_df2['col_34'].to_numpy()

    ## MEAN
    mse_difference_mean = np.mean((heatmap1_mean - heatmap2_mean) ** 2)
    mse_diffs_mean.append(mse_difference_mean)
    abs_difference_mean = np.mean(np.abs(heatmap1_mean- heatmap2_mean))
    abs_diffs_mean.append(abs_difference_mean)

    ## MEDIAN
    mse_difference_median = np.mean((heatmap1_median - heatmap2_median) ** 2)
    mse_diffs_median.append(mse_difference_median)
    abs_difference_median = np.mean(np.abs(heatmap1_median- heatmap2_median))
    abs_diffs_median.append(abs_difference_median)

    ## MAX
    mse_difference_max = np.mean((heatmap1_max - heatmap2_max) ** 2)
    mse_diffs_max.append(mse_difference_max)
    abs_difference_max = np.mean(np.abs(heatmap1_max- heatmap2_max))
    abs_diffs_max.append(abs_difference_max)



mse_diffs_mean = remove_nans(mse_diffs_mean)
abs_diffs_mean = remove_nans(abs_diffs_mean)
mse_diffs_median = remove_nans(mse_diffs_median)
abs_diffs_median = remove_nans(abs_diffs_median)
mse_diffs_max = remove_nans(mse_diffs_max)
abs_diffs_max = remove_nans(abs_diffs_max)
print(f"MSE mean +- SD: {np.mean(mse_diffs_mean):.3f} $\pm$ {np.std(mse_diffs_mean):.3f}")
print(f"Abs mean +- SD: {np.mean(abs_diffs_mean):.3f} $\pm$ {np.std(abs_diffs_mean):.3f}")

print(f"MSE median +- SD: {np.mean(mse_diffs_median):.3f} $\pm$ {np.std(mse_diffs_median):.3f}")
print(f"Abs median +- SD: {np.mean(abs_diffs_median):.3f} $\pm$ {np.std(abs_diffs_median):.3f}")

print(f"MSE max +- SD: {np.mean(mse_diffs_max):.3f} $\pm$ {np.std(mse_diffs_max):.3f}")
print(f"Abs max +- SD: {np.mean(abs_diffs_max):.3f} $\pm$ {np.std(abs_diffs_max):.3f}")